# Getting data for machine learning model

In [ ]:
import pandas as pd
import yfinance as yf
import os
import logging
from decimal import Decimal

In [ ]:
%run "Moving Average + Bollinger bands.ipynb"

In [ ]:
all_companies = pd.read_csv("../Data/Symbols_data/AmericanAndCanadianCompanies.csv")
# all_companies = all_companies[all_companies["Name"].str.contains("Common Stock")]

In [ ]:
all_companies["Sector"] = all_companies["Sector"].fillna("Other")
necessary_data = all_companies[["Symbol", "Name", "Sector"]] 
necessary_data = all_companies[(all_companies["Name"].str.contains("Common Stock")) & (all_companies["Volume"] > 100000)]
len(all_companies), len(necessary_data)

In [ ]:
necessary_data[:20]

In [ ]:
def save_downloaded_data(df):
    '''
        Downloads all data from Yahoo Finance based on the tickers inside given dataframe

        Arguments:
            df (dataframe) - data frame with desired stock symbols
    '''
    row = necessary_data.loc[necessary_data["Symbol"] == df]
    str_test = row["Symbol"].astype(str)
    label = row["Symbol"].to_string(index = False)

    directory_name = row["Sector"].to_string(index = False)
    base_path = "../Data/Stocks_data/"
    if not os.path.exists(base_path):
        os.mkdir(base_path)
    directory_path = f"../Data/Stocks_data/{directory_name}"
    if not os.path.exists(directory_path):
        os.mkdir(directory_path)
    file_name = row["Name"].to_string(index = False)
    full_path = os.path.join(directory_path, file_name).replace('\\', "/")
    
    try:
        download_result = yf.download(label, period = 'max', auto_adjust = True)
        download_result.to_csv(f'{full_path}.csv')
        print(f'{full_path} downloaded')
        test = pd.read_csv(f'{full_path}.csv')
        calculate_all_stocks_related_data(test)
        test.to_csv(f'{full_path}.csv', index = False)
    except Exception as e:
        logging.exception("An Exception was thrown!")

In [ ]:
pd.set_option("display.precision", 3)
test = necessary_data["Symbol"][:100].map(save_downloaded_data)